In [1]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.axes_grid1 import make_axes_locatable

from transform_output_format import get_2D_output, get_4D_output

pd.set_option ('display.max_columns', None)


In [4]:
# Load data from raw_data
data = np.load('../raw_data/baseline.npz', allow_pickle=True)

In [3]:
y_train = pd.read_csv('../raw_data/y_train.csv')

In [5]:
# Converted y_train(df), a vector, into a matrix
y_train.shape

(1845, 10405)

In [6]:
y_train=get_4D_output(y_train)

In [7]:
y_train.shape

(1845, 4, 51, 51)

In [8]:
GHI=data['GHI']
CLS=data['CLS']

In [9]:
X_model=np.zeros((4,4,1,51,51))
for j in range (4):
    for i in range (4):
        for z in range (1):
            if z==0:
                for k in range (51):
                    for h in range (51):
                        X_model[j,i,z,k,h]=GHI[j,i,k+15,h+15]               
            #else:
                #for k in range (51):
                    #for h in range (51):
                        #X_model[j,i,z,k,h]=CLS[j,i,k+15,h+15]
X_model.shape

(4, 4, 1, 51, 51)

In [10]:
y_model=np.zeros((4,4,1,51,51))
for j in range (4):
    for i in range (4):
        for z in range (1):
            for k in range (51):
                    for h in range (51):
                        y_model[j,i,z,k,h]=y_train[j,i,k,h]
y_model.shape

(4, 4, 1, 51, 51)

In [59]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics

def init_model(X,y):
    model = models.Sequential()

    model.add(layers.ConvLSTM2D(32, kernel_size=(3,3), padding='same',
                                data_format='channels_first',
                                return_sequences=True, input_shape=(4,1,51,51)))
    #return_sequences=True output the image at each time step
    model.add(layers.Dense(320))
    print(model.output_shape)
    model.add(layers.Reshape((4,32*51*320)))
    model.add(layers.LSTM(64, return_sequences=False))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='relu'))
    
    adam = optimizers.Adam(learning_rate=0.02)
    model.compile(loss='mse', optimizer=adam, metrics=["mae"])
    print(model.summary())
    
    return model

In [60]:
model = init_model(X_model, y_model)

(None, 4, 32, 51, 320)


2022-12-05 20:15:06.629761: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 534773760 exceeds 10% of free system memory.
2022-12-05 20:15:08.296715: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 534773760 exceeds 10% of free system memory.


Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_20 (ConvLSTM2D)  (None, 4, 32, 51, 51)    38144     
                                                                 
 dense_21 (Dense)            (None, 4, 32, 51, 320)    16640     
                                                                 
 reshape_14 (Reshape)        (None, 4, 522240)         0         
                                                                 
 lstm_3 (LSTM)               (None, 64)                133710080 
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_22 (Dense)            (None, 1)                 65        
                                                                 
Total params: 133,764,929
Trainable params: 133,764,9

In [61]:
def fit_model(model): 
    model.fit(X_model, y_model, batch_size = 32, epochs = 50)
    return model

In [62]:
model= fit_model(model)

Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node 'gradient_tape/mean_squared_error/BroadcastGradientArgs' defined at (most recent call last):
    File "/home/federico/.pyenv/versions/3.8.12/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/federico/.pyenv/versions/3.8.12/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/federico/.pyenv/versions/3.8.12/envs/solar_forecasting/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/federico/.pyenv/versions/3.8.12/envs/solar_forecasting/lib/python3.8/site-packages/traitlets/config/application.py", line 982, in launch_instance
      app.start()
    File "/home/federico/.pyenv/versions/3.8.12/envs/solar_forecasting/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/federico/.pyenv/versio